In [1]:
import numpy as np
import pandas as pd
from pandas import read_csv
from datetime import datetime
from statsmodels.tsa.seasonal import seasonal_decompose
import matplotlib
import math

In [2]:
date_parser = lambda d: datetime.strptime(d, '%Y%m')

# Read Dataset from CSV into Pandas Dataframe.
indexed_data_df = read_csv(
    '../../data/data.csv',
    names=['date', 'temperature'],
    dtype={'date': 'str', 'temperature': 'float'},
    parse_dates=['date'],
    date_parser=date_parser,
    index_col='date'
)

data_df = read_csv(
    '../../data/data.csv',
    names=['date', 'temperature'],
    dtype={'date': 'str', 'temperature': 'float'},
    parse_dates=['date'],
    date_parser=date_parser
)

In [3]:
# Isolate dataset trend only and save it as a numpy array
decompose_res = seasonal_decompose(indexed_data_df, model='additive', freq=12)
trend_data_np = np.array([[d, round(t, 5)] for d, t in zip(data_df['date'].to_numpy(), decompose_res.trend) if not math.isnan(t)])
print(trend_data_np)

[[numpy.datetime64('1880-07-01T00:00:00.000000000') -0.12625]
 [numpy.datetime64('1880-08-01T00:00:00.000000000') -0.12375]
 [numpy.datetime64('1880-09-01T00:00:00.000000000') -0.11375]
 ...
 [numpy.datetime64('2019-12-01T00:00:00.000000000') 1.01125]
 [numpy.datetime64('2020-01-01T00:00:00.000000000') 1.00917]
 [numpy.datetime64('2020-02-01T00:00:00.000000000') 1.00917]]


In [6]:
from sklearn.svm import LinearSVR
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

linear_svr_model = make_pipeline(StandardScaler(), LinearSVR())
linear_svr_model.fit(trend_data_np[:,0].reshape((-1, 1)), trend_data_np[:,1])

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('linearsvr', LinearSVR())])

In [24]:
from datetime import date
from dateutil import relativedelta

# Generate Future Dates
np_future_dates = np.array()
curr_date = datetime.today().replace(day=1, hour=0, minute=0, second=0, microsecond=0)
while curr_date <= datetime.strptime('01-2050', '%m-%Y'):
    curr_date += relativedelta.relativedelta(months=1)
    curr_date.replace(day=1, hour=0, minute=0, second=0, microsecond=0)
    np_future_dates.append(np.datetime64(curr_date))
np_future_dates = np.array(np_future_dates)

# linear_svr_model.predict()

AttributeError: 'numpy.ndarray' object has no attribute 'append'